Продолжим работу с данными, которые были использованы в ДЗ 2 и 3, продолжим решать задачу обнаружения мошеннических транзакций, что позволит получить полное решение задачи / полный пайплайн.

Задание 0: выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

Задание 1: признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

Задание 2: сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2

Рассматривать их как категориальных признаки.

Задание 3: Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

Задание 4: Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

Задание 5: Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

Задание 6: выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

Задание 7 (опция): выполнить предварительную подготовку / очистку признаков P_emaildomain и R_emaildomain (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.

In [1]:
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn import metrics
import seaborn as sns
import warnings
from tqdm import tqdm
from typing import List, Tuple
from scipy.stats import ttest_rel
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from classes_and_functions import bootstrap_calculate_confidence_interval, make_cross_validation,make_cross_validation_gr

warnings.simplefilter("ignore")
from sklearn.model_selection import TimeSeriesSplit
%matplotlib inline

In [3]:
train= pd.read_csv('train.csv')

In [4]:
train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,3166995,0,3958217,39.00,W,1877,310.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179996,3166996,0,3958237,59.95,W,10075,514.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179997,3166997,0,3958241,34.00,W,6053,122.0,150.0,mastercard,195.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179998,3166998,0,3958260,59.00,W,7726,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
train.set_index('TransactionID',inplace=True)

In [6]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180000 entries, 2987000 to 3166999
Columns: 393 entries, isFraud to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 541.1+ MB


In [8]:
object_features = train.select_dtypes(include=[np.object]).columns
train[object_features]= train[object_features].astype('category')

In [9]:
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])

In [10]:
scors = pd.DataFrame()

### Задание 0: выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага

### Базовая модель

In [11]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    'num_leaves':45,
    'max_depth':4,
    'learning_rate':0.1,
    'n_estimators':5000,
    "metric": "auc",
    "n_jobs": 15,
    'reg_alpha':2,
    'reg_lambda':2,
    "random_state": 30,
#     'device':"gpu"
}

In [12]:
model = lgb.LGBMClassifier(**params)

In [13]:
%%time
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['base','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 31.5 s


(1967,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9965549735307786)]),
              'valid_1': OrderedDict([('auc', 0.9527215435494588)])}))

### Задание 1: признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [14]:
train['TransactionDT'] =  pd.to_datetime('2017-12-01') + pd.to_timedelta(train.TransactionDT,unit='S')

год, месяц, день недели, час, день.

In [15]:
object_features= object_features.to_list()
object_features.append('TransactionDT')

In [16]:
train['year'] = train.TransactionDT.apply(lambda x: x.year)

In [17]:
train['month'] = train.TransactionDT.apply(lambda x: x.month)

In [18]:
train['day'] =  train.TransactionDT.apply(lambda x: x.day)

In [19]:
train['hour'] = train.TransactionDT.apply(lambda x: x.hour)

In [20]:
train['weekday'] = train.TransactionDT.apply(lambda x: x.weekday())

In [21]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 90,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['TransactionDT','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 45.9 s


(2707,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.998940405914203)]),
              'valid_1': OrderedDict([('auc', 0.9507593011596603)])}))

Скор на валидации упал

### Задание 2: сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2

In [22]:
train['card1_card2'] = train.apply(lambda x: str(x.card1)+'_' + str(x.card2),axis=1)

In [23]:
train['card1_card2_card_3_card_5'] =\
    train.apply(lambda x: str(x.card1_card2)+'_' + str(x.card3) + '_'+str(x.card4) + '_'+ str(x.card5) ,axis=1)

In [24]:
train['card1_card2_card_3_card_5_addr1_addr2'] =\
    train.apply(lambda x: str(x.card1_card2_card_3_card_5)+'_' + str(x.addr1) + '_'+str(x.addr2) ,axis=1)

In [25]:

object_features = object_features + ['card1_card2','card1_card2_card_3_card_5','card1_card2_card_3_card_5_addr1_addr2']

In [26]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['concate cards','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 19.1 s


(935,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9956111496620772)]),
              'valid_1': OrderedDict([('auc', 0.9467338184054748)])}))

скор на валидации упал

#### Задание 3: Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

In [27]:
for i in [ 'card1', 'card2', 'card3','card5' ,'card6', 'addr1', 'addr2']:
    train[i] = train[i].map(train[i].value_counts())

In [28]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['FrequencyEncoder','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 19.4 s


(938,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9977382164445812)]),
              'valid_1': OrderedDict([('auc', 0.9478016265152199)])}))

### Задание 4: Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [29]:
train.TransactionAmt

TransactionID
2987000     68.50
2987001     29.00
2987002     59.00
2987003     50.00
2987004     50.00
            ...  
3166995     39.00
3166996     59.95
3166997     34.00
3166998     59.00
3166999    226.00
Name: TransactionAmt, Length: 180000, dtype: float64

In [30]:
for i in [ 'card1', 'card2', 'card3','card5' ,'card6', 'addr1', 'addr2']:
    name = 'TransactionAmt_mean_'+str(i)
    train[name] = train[i].map(train.groupby(i)['TransactionAmt'].mean())
    train[name] = train[name].astype(np.float64)
    train[name] = train.TransactionAmt/train[name] 
    name = 'TransactionAmt_std_'+str(i)
    train[name] = train[i].map(train.groupby(i)['TransactionAmt'].std())
    train[name] = train[name].astype(np.float64)
    train[name] = train.TransactionAmt/train[name] 
    
    


In [31]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['TransactionAmt_stats','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 19.7 s


(919,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9979416803112372)]),
              'valid_1': OrderedDict([('auc', 0.9483070336202258)])}))

### Задание 5: Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [32]:
for i in [ 'card1', 'card2', 'card3','card5' ,'card6', 'addr1', 'addr2']:
    name = 'D15_mean_'+str(i)
    train[name] = train[i].map(train.groupby(i)['D15'].mean())
    train[name] = train[name].astype(np.float64)
    train[name] = train.TransactionAmt/train[name] 
    name = 'D15_std_'+str(i)
    train[name] = train[i].map(train.groupby(i)['D15'].std())
    train[name] = train[name].astype(np.float64)
    train[name] = train.TransactionAmt/train[name] 
    

In [33]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['D15_stats','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 21.6 s


(974,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9982298205610648)]),
              'valid_1': OrderedDict([('auc', 0.9482703728296701)])}))

### Задание 6: выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

In [34]:
train['TransactionAmt_int'] = train.TransactionAmt//1
train['TransactionAmt_float'] = train.TransactionAmt%1
train['TransactionAmt_log'] = np.log(train.TransactionAmt+0.01)

In [35]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['TransactionAmt_new','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 24.9 s


(1171,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9989245044336792)]),
              'valid_1': OrderedDict([('auc', 0.9499363526396437)])}))

### Задание 7 (опция): выполнить предварительную подготовку / очистку признаков P_emaildomain и R_emaildomain (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.

In [36]:
train.P_emaildomain = train.P_emaildomain.map(train.P_emaildomain.value_counts())

In [37]:
train.P_emaildomain = train.P_emaildomain.map(train.P_emaildomain.value_counts())

In [38]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['emaildomain_freq','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 23.3 s


(1065,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9986646301995645)]),
              'valid_1': OrderedDict([('auc', 0.9496490188259558)])}))

In [39]:
scors

,auc
base,0.952722
TransactionDT,0.950759
concate cards,0.946734
FrequencyEncoder,0.947802
TransactionAmt_stats,0.948307
D15_stats,0.948270
TransactionAmt_new,0.949936
emaildomain_freq,0.949649


## Ранее мы накладывали новые признаки друг на друга. Теперь попробуем считать скор раздельно по каждому признаку

### №1

In [40]:
%%time
train= pd.read_csv('train.csv')
train.set_index('TransactionID',inplace=True)
object_features = train.select_dtypes(include=[np.object]).columns
train[object_features]= train[object_features].astype('category')
train['TransactionDT'] =  pd.to_datetime('2017-12-01') + pd.to_timedelta(train.TransactionDT,unit='S')
object_features= object_features.to_list()
object_features.append('TransactionDT')
train['year'] = train.TransactionDT.apply(lambda x: x.year)
train['month'] = train.TransactionDT.apply(lambda x: x.month)
train['day'] =  train.TransactionDT.apply(lambda x: x.day)
train['hour'] = train.TransactionDT.apply(lambda x: x.hour)
train['weekday'] = train.TransactionDT.apply(lambda x: x.weekday())
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 90,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['TransactionDT','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_


Wall time: 55.7 s


(2707,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.998940405914203)]),
              'valid_1': OrderedDict([('auc', 0.9507593289751312)])}))

### №2

In [41]:
%%time
train= pd.read_csv('train.csv')
train.set_index('TransactionID',inplace=True)
object_features = train.select_dtypes(include=[np.object]).columns.to_list()
train[object_features]= train[object_features].astype('category')
train['card1_card2'] = train.apply(lambda x: str(x.card1)+'_' + str(x.card2),axis=1)
train['card1_card2_card_3_card_5'] =\
    train.apply(lambda x: str(x.card1_card2)+'_' + str(x.card3) + '_'+str(x.card4) + '_'+ str(x.card5) ,axis=1)
train['card1_card2_card_3_card_5_addr1_addr2'] =\
    train.apply(lambda x: str(x.card1_card2_card_3_card_5)+'_' + str(x.addr1) + '_'+str(x.addr2) ,axis=1)

object_features = object_features + ['card1_card2','card1_card2_card_3_card_5','card1_card2_card_3_card_5_addr1_addr2']
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['concate cards','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 45 s


(1480,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.998448561945898)]),
              'valid_1': OrderedDict([('auc', 0.9503510813097393)])}))

### № 3

In [42]:
%%time
train= pd.read_csv('train.csv')
train.set_index('TransactionID',inplace=True)
object_features = train.select_dtypes(include=[np.object]).columns.to_list()
train[object_features]= train[object_features].astype('category')
for i in [ 'card1', 'card2', 'card3','card5' ,'card6', 'addr1', 'addr2']:
    train[i] = train[i].map(train[i].value_counts())
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['FrequencyEncoder','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_    
    

Wall time: 33.7 s


(1821,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9958435452201528)]),
              'valid_1': OrderedDict([('auc', 0.9504295209374837)])}))

### № 4

In [43]:
%%time
train= pd.read_csv('train.csv')
train.set_index('TransactionID',inplace=True)
object_features = train.select_dtypes(include=[np.object]).columns.to_list()
train[object_features]= train[object_features].astype('category')
for i in [ 'card1', 'card2', 'card3','card5' ,'card6', 'addr1', 'addr2']:
    name = 'TransactionAmt_mean_'+str(i)
    train[name] = train[i].map(train.groupby(i)['TransactionAmt'].mean())
    train[name] = train[name].astype(np.float64)
    train[name] = train.TransactionAmt/train[name] 
    name = 'TransactionAmt_std_'+str(i)
    train[name] = train[i].map(train.groupby(i)['TransactionAmt'].std())
    train[name] = train[name].astype(np.float64)
    train[name] = train.TransactionAmt/train[name] 
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['TransactionAmt_stats','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_    

Wall time: 40.2 s


(2056,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9986813016391954)]),
              'valid_1': OrderedDict([('auc', 0.9502662719391745)])}))

### № 5

In [44]:
%%time
train= pd.read_csv('train.csv')
train.set_index('TransactionID',inplace=True)
object_features = train.select_dtypes(include=[np.object]).columns.to_list()
train[object_features]= train[object_features].astype('category')
for i in [ 'card1', 'card2', 'card3','card5' ,'card6', 'addr1', 'addr2']:
    name = 'D15_mean_'+str(i)
    train[name] = train[i].map(train.groupby(i)['D15'].mean())
    train[name] = train[name].astype(np.float64)
    train[name] = train.TransactionAmt/train[name] 
    name = 'D15_std_'+str(i)
    train[name] = train[i].map(train.groupby(i)['D15'].std())
    train[name] = train[name].astype(np.float64)
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['D15_stats','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_    

Wall time: 38.4 s


(1835,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9981319471135564)]),
              'valid_1': OrderedDict([('auc', 0.9515854206433518)])}))

### № 6 

In [45]:
%%time
train= pd.read_csv('train.csv')
train.set_index('TransactionID',inplace=True)
object_features = train.select_dtypes(include=[np.object]).columns.to_list()
train[object_features]= train[object_features].astype('category')
train['TransactionAmt_int'] = train.TransactionAmt//1
train['TransactionAmt_float'] = train.TransactionAmt%1
train['TransactionAmt_log'] = np.log(train.TransactionAmt+0.01)
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['TransactionAmt_new','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_


Wall time: 22.7 s


(1030,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9878944943493526)]),
              'valid_1': OrderedDict([('auc', 0.9457054248178977)])}))

### № 7

In [46]:
%%time
train= pd.read_csv('train.csv')
train.set_index('TransactionID',inplace=True)
object_features = train.select_dtypes(include=[np.object]).columns.to_list()
train[object_features]= train[object_features].astype('category')
train.P_emaildomain = train.P_emaildomain.map(train.P_emaildomain.value_counts())
train.P_emaildomain = train.P_emaildomain.map(train.P_emaildomain.value_counts())
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['emaildomain_freq','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 28.7 s


(1450,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9933170156577581)]),
              'valid_1': OrderedDict([('auc', 0.9487687426205555)])}))

In [47]:
scors

,auc
base,0.952722
TransactionDT,0.950759
concate cards,0.950351
FrequencyEncoder,0.950430
TransactionAmt_stats,0.950266
D15_stats,0.951585
TransactionAmt_new,0.945705
emaildomain_freq,0.948769


Как видно ни один новый признак как вместе та ки по отдельности не увеличил качество моджели на валидации